# HAMR-Orchestra

* Frank Zalkow, Christof Weiß
* HAMR 2017, Suzhou

Imports:

In [ ]:
import os

import numpy as np
from scipy import signal
from scipy.interpolate import interp1d
import soundfile as sf

from IPython.display import Audio, display

Define file names

In [ ]:
file_orchestra = "beethoven_op67-01_orchester.wav"
file_piano = "beethoven_op67-01_klavier-streched.wav"

Read files:

In [ ]:
x_orchestra, fs_orchestra = sf.read(file_orchestra)
x_piano, fs_piano = sf.read(file_piano)
assert fs_orchestra == fs_piano
fs = fs_orchestra

display(Audio(x_orchestra, rate=fs))
display(Audio(x_piano, rate=fs))

Functions for doing HPRSS:

In [ ]:
def horizontal_median_filter(x, filter_len):
    """ x:          Input signal
        filter_len: filter length in samples
    """
    return signal.medfilt(x, [1, filter_len])


def vertical_median_filter(x, filter_len):
    """ x:          Input signal
        filter_len: filter length in samples
    """
    return signal.medfilt(x, [filter_len, 1])


def L_h(t, Fs, H):
    """ t:  time span in seconds, to be converted in samples
        Fs: sample rate
        H:  hop size
    """
    return int(np.ceil(t * Fs/H))


def L_p(d, Fs, N):
    """ d:  frequency span in hertz, to be converted in bins
        Fs: sample rate
        N:  Frame Size
    """
    return int(np.ceil(d * N / Fs))


def HPRSS(x, N, H, w, Fs, lh_sec, lp_Hz, alpha):
    """ x:      Input signal
        N:      Frame length
        H:      Hopsize
        w:      Window function of length N
        fs:     Sampling rate of x
        lh_sec: Horizontal median filter length given in seconds
        lp_Hz:  Percussive median filter length given in Hertz
        alpha:  separation factor for the residual """

    # stft
    f, t, X = signal.stft(x, Fs, w, N, N-H)

    # power spectrogram
    Y = np.abs(X) ** 2

    # median filtering
    lh_idx = L_h(lh_sec, Fs, H)
    lp_idx = L_p(lp_Hz, Fs, N)
    lh_idx -= ((lh_idx+1) % 2)
    lp_idx -= ((lp_idx+1) % 2)
    
    Y_h = horizontal_median_filter(Y, lh_idx)
    Y_p = vertical_median_filter(Y, lp_idx)

    # masking
    M_h = np.int8(Y_h >= alpha * Y_p)
    M_p = np.int8(alpha * Y_h < Y_p)
    M_r = np.logical_not(np.logical_or(M_h, M_p))
    X_h = X * M_h
    X_p = X * M_p
    X_r = X * M_r

    # istft
    t, x_h = signal.istft(X_h, Fs, w, N, N-H)
    t, x_p = signal.istft(X_p, Fs, w, N, N-H)
    t, x_r = signal.istft(X_r, Fs, w, N, N-H)

    return x_h, x_p, x_r

Functions for doing a simple dynamic range compression:

In [ ]:
def apply_transfer(signal, transfer, interpolation='linear'):
    constant = np.linspace(-1, 1, len(transfer))
    interpolator = interp1d(constant, transfer, interpolation)
    return interpolator(signal)

# smooth compression: if factor is small, its near linear, the bigger it is the
# stronger the compression
def arctan_compressor(x, factor=2):
    constant = np.linspace(-1, 1, 1000)
    transfer = np.arctan(factor * constant)
    transfer /= np.abs(transfer).max()
    return apply_transfer(x, transfer)

Apply HPRSS on Orchestra:

In [ ]:
N = 4096
H = N / 2
w = 'hann'
lh_sec = 1.0
lp_Hz = 200
alpha = 2.0

x_h_orchestra, x_p_orchestra, x_r_orchestra = HPRSS(x_orchestra, N, H, w, fs, lh_sec, lp_Hz, alpha)

display(Audio(x_h_orchestra, rate=fs))
display(Audio(x_p_orchestra, rate=fs))
display(Audio(x_r_orchestra, rate=fs))

sf.write(os.path.join('out', os.path.splitext(file_orchestra)[0] + '_h.wav'), x_h_orchestra, fs)
sf.write(os.path.join('out', os.path.splitext(file_orchestra)[0] + '_p.wav'), x_p_orchestra, fs)
sf.write(os.path.join('out', os.path.splitext(file_orchestra)[0] + '_r.wav'), x_r_orchestra, fs)

Apply HPRSS on piano:

In [ ]:
N = 4096
H = N / 2
w = 'hann'
lh_sec = 1.0
lp_Hz = 50
alpha = 3.0

x_h_piano, x_p_piano, x_r_piano = HPRSS(x_piano, N, H, w, fs, lh_sec, lp_Hz, alpha)

display(Audio(x_h_piano, rate=fs))
display(Audio(x_p_piano, rate=fs))
display(Audio(x_r_piano, rate=fs))

sf.write(os.path.join('out', os.path.splitext(file_piano)[0] + '_h.wav'), x_h_piano, fs)
sf.write(os.path.join('out', os.path.splitext(file_piano)[0] + '_p.wav'), x_p_piano, fs)
sf.write(os.path.join('out', os.path.splitext(file_piano)[0] + '_r.wav'), x_r_piano, fs)

Some normalization and compression:

In [ ]:
x_p_piano /= np.max(np.abs(x_p_piano))
x_p_piano_comp = arctan_compressor(x_p_piano, 2.0)
x_p_piano_comp /= np.max(np.abs(x_p_piano_comp))
x_p_piano_comp = x_p_piano_comp
x_h_orchestra /= np.max(np.abs(x_h_orchestra))
display(Audio(x_p_piano_comp, rate=fs))

In [ ]:
x_sum = 0.7 * x_p_piano_comp + x_h_orchestra
x_sum /= np.max(np.abs(x_sum))
display(Audio(x_sum, rate=fs))
sf.write(os.path.join('out', os.path.splitext(file_piano)[0] + '_--_' + os.path.splitext(file_orchestra)[0] + '.wav'), x_sum, fs)